In [1]:
import os
import re
from collections import Counter
import json
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
cm = 1/2.54

# force GPU device
os.environ["CUDA_VISIBLE_DEVICES"]='1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

from keras.src.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.src.layers import SimpleRNN, LSTM, Dense, Dropout, Bidirectional, Embedding, Input, RepeatVector, TimeDistributed, Reshape
from keras import Sequential, Model
from keras.losses import CategoricalCrossentropy, CategoricalFocalCrossentropy, SparseCategoricalCrossentropy
from keras.optimizers import Adam
from keras.models import load_model
from keras.metrics import CategoricalAccuracy, TopKCategoricalAccuracy

from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import Sequence

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

from utils import *

In [2]:
data_directory = '/home/jrosendahl/datasets/cadets/sequences_export_benign_filetypes_path_ts/'
experiment_name = 'path_autoencoder_v2'

checkpoint_path = f'saves/{experiment_name}'
model_path = f'{checkpoint_path}/model.keras'
log_path = f'{checkpoint_path}/log.csv'
history_path = f'{checkpoint_path}/history.npy'

# ensure directory exists
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)

In [3]:
# list of limitations for paths:
# allowed charaters: alphanumeric chars + '/.'
# all chars are transformed to lowercase
def preprocess_path(path: str) -> str:
    path = path.lower()
    path = re.sub(r'[^a-z0-9/._]', '', path)
    return path

In [4]:
# load data

vocab = set()
X = []
longest_path = 0
files_loaded = 0

for filename in os.listdir(data_directory):
    with open(os.path.join(data_directory, filename), 'r') as f:
        files_loaded += 1
        if files_loaded % 50000 == 0:
            print(f"Files loaded: {files_loaded}")

        for line in f:
            line = line.split(',')
            path1 = line[4]
            path2 = line[5]


            for path in [path1, path2]:
                if path in ['', 'None']:
                    continue
                path = preprocess_path(path1)
                vocab.update(path)
                X.append(path)


mean_length = np.mean([len(path) for path in X])
vocab_size = len(vocab)

char_to_idx = {char: idx+1 for idx, char in enumerate(vocab)}
# add padding character
char_to_idx[''] = 0
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

print(f"Files loaded: {files_loaded}")
print(f'{len(X)=}')
print(f'{vocab_size=}')
print(f'{longest_path=}')
print(f'{mean_length=}')

Files loaded: 50000
Files loaded: 100000
Files loaded: 150000
Files loaded: 200000
Files loaded: 215150
len(X)=8161337
vocab_size=39
longest_path=0
mean_length=16.37407400282576


In [5]:
fixed_length = 50

def vectorize_data(X, char_to_idx, fixed_length):
    X_vec = np.zeros((len(X), fixed_length), dtype=np.int32)
    for i, path in enumerate(X):
        for j, char in enumerate(path):
            if j >= fixed_length:
                break
            X_vec[i, j] = char_to_idx[char]
    return X_vec

def vectorized_to_string(X, idx_to_char):
    return [''.join([idx_to_char[idx] for idx in path]) for path in X]

X_vec = vectorize_data(X, char_to_idx, fixed_length)

In [6]:
print(f'{X_vec.shape=}')
print(f'{X_vec[0]=}')

X_vec.shape=(8161337, 50)
X_vec[0]=array([ 1, 33, 18, 39,  1, 28, 36, 14,  1,  2,  7,  3,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)


In [7]:
class DataGenerator(Sequence):
    def __init__(self, data, batch_size, fixed_length, vocab_size, shuffle=True):
        # call super
        super(DataGenerator, self).__init__()
        self.data = data
        self.batch_size = batch_size
        self.fixed_length = fixed_length
        self.vocab_size = vocab_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.data))
        self.on_epoch_end()

    def __len__(self):
        # Number of batches per epoch
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        # Generate indices for the batch
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Generate data for the batch
        X_batch = self.__data_generation(batch_indices)

        # Since it's an autoencoder, the target data is the same as the input data
        return X_batch, X_batch

    def on_epoch_end(self):
        # Shuffle indices after each epoch if shuffle is True
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __data_generation(self, batch_indices):
        # Generate data for a batch of given indices
        # This is where you can customize your data loading and processing
        batch_data = [self.data[i] for i in batch_indices]

        # Convert the batch data to a numpy array (or whatever format is needed)
        X_batch = np.array(batch_data)

        # Ensure the data is padded or truncated to fixed_length
        X_batch = np.array([np.pad(x, (0, max(0, self.fixed_length - len(x))), constant_values=0)[:self.fixed_length] for x in X_batch])

        return X_batch

In [8]:
# prep data generators
batch_size = 512
train_size = int(0.8 * len(X_vec))
val_size = int(0.15 * len(X_vec))
test_size = len(X_vec) - train_size - val_size

train_gen = DataGenerator(X_vec[:train_size], batch_size, fixed_length, vocab_size)
val_gen = DataGenerator(X_vec[train_size:train_size+val_size], batch_size, fixed_length, vocab_size)
test_gen = DataGenerator(X_vec[train_size+val_size:], batch_size, fixed_length, vocab_size)

x_gen = DataGenerator(X_vec, batch_size, fixed_length, vocab_size)

In [23]:
# build model

latent_dim = 64
hidden_dim = 128
hidden_layers = 2

def build_autoencoder(vocab_size, fixed_length, latent_dim, hidden_dim, recurrent_layers):
    encoder_input = Input(shape=(fixed_length,))
    x = Embedding(input_dim=vocab_size+1, output_dim=hidden_dim)(encoder_input)

    for i in range(recurrent_layers):
        if i == recurrent_layers - 1:
            # Last LSTM layer should not return sequences
            x = LSTM(hidden_dim, return_sequences=False)(x)
        else:
            x = LSTM(hidden_dim, return_sequences=True)(x)

    encoder_output = Dense(latent_dim, activation='relu')(x)

    # Create the encoder model
    encoder = Model(encoder_input, encoder_output, name='encoder')

    # Decoder definition
    decoder_input = Input(shape=(latent_dim,))
    x = Dense(hidden_dim, activation='relu')(decoder_input)
    x = RepeatVector(fixed_length)(x)
    for i in range(recurrent_layers):
        x = LSTM(hidden_dim, return_sequences=True)(x)

    decoder_output = TimeDistributed(Dense(vocab_size+1, activation='softmax'))(x)

    # Create the decoder model
    decoder = Model(decoder_input, decoder_output, name='decoder')

    # Autoencoder definition
    autoencoder_input = encoder_input
    encoded_sequence = encoder(autoencoder_input)
    decoded_sequence = decoder(encoded_sequence)

    # Create the autoencoder model by combining encoder and decoder
    autoencoder = Model(autoencoder_input, decoded_sequence, name='autoencoder')

    autoencoder.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return encoder, decoder, autoencoder

In [24]:
encoder, decoder, autoencoder = build_autoencoder(
    vocab_size=vocab_size,        # Example vocab size
    fixed_length=fixed_length,      # Example fixed length
    latent_dim=48,        # Latent space dimension
    hidden_dim=128,       # Number of neurons in hidden layers
    recurrent_layers=3     # Number of LSTM layers in encoder and decoder
)

encoder.summary()
decoder.summary()
autoencoder.summary()


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_4 (Embedding)         │ (None, 50, 128)        │         5,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_27 (LSTM)                  │ (None, 50, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_28 (LSTM)                  │ (None, 50, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_29 (LSTM)                  │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 48)             │         6,192 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 406,064 (1.55 MB)

 Trainable params: 406,064 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

...

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ (None, 48)             │       406,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 50, 40)         │       406,184 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 812,248 (3.10 MB)

 Trainable params: 812,248 (3.10 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        verbose=1,
        restore_best_weights=True
    ),
    ModelCheckpoint(
        model_path,
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        verbose=1
    ),
    CSVLogger(log_path)
]

# metrics
metrics = [
    CategoricalAccuracy(),
    TopKCategoricalAccuracy(k=3),
]

In [25]:
# training
history = autoencoder.fit(
    train_gen,
    validation_data=val_gen,
    epochs=1,
    callbacks=callbacks,
    verbose=1,
)

# save model
encoder.save(f'{checkpoint_path}/encoder.keras')
decoder.save(f'{checkpoint_path}/decoder.keras')
autoencoder.save(f'{checkpoint_path}/autoencoder.keras')

  121/12752 ━━━━━━━━━━━━━━━━━━━━ 8:46 42ms/step - accuracy: 0.6840 - loss: 1.5852

KeyboardInterrupt: 

In [ ]:
log = pd.read_csv(log_path)

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20*cm, 15*cm))

ax1.plot(log['epoch'], log['categorical_accuracy'], label='categorical_accuracy', color='red')
ax1.plot(log['epoch'], log['val_categorical_accuracy'], label='val_categorical_accuracy', color='orange')
ax1.plot(log['epoch'], log[f'top_3_categorical_accuracy'], label=f'top_3_categorical_accuracy', color='royalblue')
ax1.plot(log['epoch'], log[f'val_top_3_categorical_accuracy'], label=f'val_top_3_categorical_accuracy', color='navy')
ax1.set_title('training metrics')
ax1.set_xlabel('epoch')
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=1)  # Legend below the plot

ax2.plot(log['epoch'], log['loss'], label='loss')
ax2.plot(log['epoch'], log['val_loss'], label='val_loss')
# plot learning rate with different scale, show on right side
ax2b = ax2.twinx()
ax2b.plot(log['epoch'], log['learning_rate'], label='learning_rate', color='green', alpha=.4)
ax2.set_title('loss')
ax2.set_xlabel('epoch')
ax2.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=2)  # Legend below the plot
ax2b.legend(loc='upper center', bbox_to_anchor=(0.5, -0.3), ncol=1)  # Legend below the plot

fig.tight_layout(rect=[0, 0.1, 1, 1])

plt.savefig(f'{checkpoint_path}/metrics_loss.svg')
plt.show()

In [ ]:
# evaluate model on test data
test_loss = autoencoder.evaluate(test_gen, verbose=2)

print(f'Test loss: {test_loss}')